In [1]:
#pip install "unstructured[pdf]"

In [1]:
# import argparse
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
# from langchain_core.messages import HumanMessage, SystemMessage
# from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain_huggingface import HuggingFaceEmbeddings
# from langchain_google_vertexai import VertexAIEmbeddings
from langchain_cohere import CohereEmbeddings
from langchain_chroma import Chroma
# import getpass
import os
import model as md
from dotenv import load_dotenv, find_dotenv
import logging

In [2]:
load_dotenv(override=True)
DATA_PATH = "data/text_db/raw"
CHROMA_PATH = "data/text_db/chroma_vertex"
chroma_db = None
groq_api_key = os.getenv("GROQ_API_KEY")
cohere_api_key = os.getenv("COHERE_API_KEY")
model = ChatGroq(model="llama3-8b-8192")
# logging.basicConfig(level=logging.DEBUG)  # Nivel global
# logger = logging.getLogger("chromadb")    # Logger específico para Chroma
# logger.setLevel(logging.DEBUG)

### TODO: Some warnings from here and tensorflow usage. Check if something else can be done.
# embeddings = CohereEmbeddings(model="embed-english-v3.0")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

### TODO: It doesn't read very well some characters. Try another pdf reader, such as llamaparse, pypdf2 or pdfplumber.
def load_split_documents():
    text_loader_kwargs={'autodetect_encoding': True}
    loader = DirectoryLoader(DATA_PATH, glob="*.pdf", loader_kwargs=text_loader_kwargs)
    chunks = loader.load_and_split(RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500, add_start_index=True)) ### We do the splitting in the same def as the loading.
    return chunks

c:\Users\raulg\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 

In [ ]:
### IT WORKS
chunks = md.load_split_documents()
chroma_db = md.chroma_read(chunks,embeddings)
chunks[100].page_content

Local Chroma DB not found. Creating Chroma database...


In [10]:
chunks = load_split_documents()
chunks[100].page_content

'Iteration – if we have 10,000 images as data and a batch size of 200. then an epoch should run 50 iterations (10,000 divided\n\nby 50).\n\nQ80. What Are the Di\x00erent Layers on CNN?\n\nThere are four layers in CNN:\n\n1. Convolutional Layer – the layer that performs a convolutional operation, creating several smaller picture windows to go\n\nover the data.\n\n2. ReLU Layer – it brings non-linearity to the network and converts all the negative pixels to zero. The output is a recti\x00ed\n\nfeature map.\n\n3. Pooling Layer – pooling is a down-sampling operation that reduces the dimensionality of the feature map.\n\n4. Fully Connected Layer – this layer recognizes and classi\x00es the objects in the image.\n\nQ81. What Is Pooling on CNN, and How Does It Work?\n\nPooling is used to reduce the spatial dimensions of a CNN. It performs down-sampling operations to reduce the dimensionality\n\nand creates a pooled feature map by sliding a \x00lter matrix over the input matrix.\n\nQ82. What a

In [11]:
def chroma_create(chunks, embeddings):
    print("Creating Chroma database...")
    chroma_db = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)
    print(f"Saved {len(chunks)} to {CHROMA_PATH}")
    return chroma_db

In [5]:
os.path.exists(CHROMA_PATH+'/chroma.sqlite3')

False

In [12]:
def chroma_read(chunks, embeddings):
    if os.path.exists(CHROMA_PATH+'/chroma.sqlite3') == True:
        print("Local Chroma DB found. Reading Chroma database...")
        chroma_db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
        print("Loaded Chroma DB from disk.")
    else:
        print("Local Chroma DB not found. Creating Chroma database...")
        chroma_db = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)
        print(f"Saved {len(chunks)} to {CHROMA_PATH}")
    return chroma_db

In [13]:
chroma_db = chroma_read(chunks, embeddings)

Local Chroma DB found. Reading Chroma database...
Loaded Chroma DB from disk.


In [16]:
query_text = "What is Data Science? List the di􀃠erences between supervised and unsupervised learning." ### TODO: Change when turning into a function.
results = chroma_db.similarity_search(query_text)
print(query_text)
print("="*50)
print(results)

What is Data Science? List the di􀃠erences between supervised and unsupervised learning.
[Document(metadata={'source': 'data\\text_db\\raw\\100_ds_interview_questions.pdf', 'start_index': 1060}, page_content='Deep Learning Interview Questions\n\nBefore moving ahead, you may go through the recording of Data Science Interview Questions where our instructor has shared\n\nhis experience and expertise that will help you to crack any Data Science.\n\nData Science Interview Questions | Edureka\n\nData Science Interview Questions | Data Science Tutorial | Data Science I Data Science Interview Questions | Data Science Tutorial | Data Science I\n\nBASIC DATA SCIENCE INTERVIEW QUESTIONS\n\nQ1. What is Data Science? List the di\x00erences between supervised and unsupervised learning.\n\nData Science is a blend of various tools, algorithms, and machine learning principles with the goal to discover hidden patterns\n\nfrom the raw data. How is this di\x00erent from what statisticians have been doing f

In [8]:
results

[(Document(metadata={'source': 'data\\text_db\\raw\\100_ds_interview_questions.pdf', 'start_index': 1060}, page_content='Deep Learning Interview Questions\n\nBefore moving ahead, you may go through the recording of Data Science Interview Questions where our instructor has shared\n\nhis experience and expertise that will help you to crack any Data Science.\n\nData Science Interview Questions | Edureka\n\nData Science Interview Questions | Data Science Tutorial | Data Science I Data Science Interview Questions | Data Science Tutorial | Data Science I\n\nBASIC DATA SCIENCE INTERVIEW QUESTIONS\n\nQ1. What is Data Science? List the di\x00erences between supervised and unsupervised learning.\n\nData Science is a blend of various tools, algorithms, and machine learning principles with the goal to discover hidden patterns\n\nfrom the raw data. How is this di\x00erent from what statisticians have been doing for years?\n\nThe answer lies in the di\x00erence between explaining and predicting.\n\n

In [19]:
Model = ChatGroq(model="llama3-8b-8192")
prompt_template = ChatPromptTemplate.from_template("""
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
""")
prompt = prompt_template.format(context=context_text, question=query_text)
response = model.invoke(prompt)
print("Question: ",query_text)
print("-"*10)
print(response.content)

Question:  What is Data Science? List the di􀃠erences between supervised and unsupervised learning.
----------
Data Science is a blend of various tools, algorithms, and machine learning principles with the goal to discover hidden patterns from the raw data. The differences between supervised and unsupervised learning are as follows:

* Supervised Learning:
	+ Input data is labelled.
	+ Uses a training data set.
	+ Used for prediction.
	+ Enables classification and regression.
* Unsupervised Learning:
	+ Input data is unlabelled.
	+ Uses the input data set.
	+ Used for analysis.
	+ Enables classification, density estimation, and dimension reduction.


# Pruebas desde model.py

In [3]:
### TODO: does it work when it has to create a new chroma db?
# chroma_db = md.chroma_read()
current_question = "What is Data Science? List the di􀃠erences between supervised and unsupervised learning."

In [5]:
expl = md.question_explanation(current_question,embeddings)
print(expl)

Searching for similar questions...
Creating prompt...
Making chain...
Invoking explanation...
Storing answer in SQL...
**What is Data Science?**

Data Science is a blend of various tools, algorithms, and machine learning principles with the goal to discover hidden patterns from raw data. It's different from what statisticians have been doing for years because data science focuses on predicting and discovering patterns, whereas statistics focuses on explaining and describing data.

**Supervised Learning vs. Unsupervised Learning**

In a job interview, it's essential to understand the differences between supervised and unsupervised learning, as they are fundamental concepts in data science.

**Supervised Learning**

* Input data is labelled, meaning the output variable is already known.
* Uses a training data set to learn from.
* Used for prediction, enabling classification and regression.
* Examples: fruit classifier (e.g., "this is an orange, this is an apple, and this is a banana").



In [2]:
# import argparse
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_groq import ChatGroq
# from langchain_core.messages import HumanMessage, SystemMessage
# from langchain_core.documents import Document
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain.document_loaders import DirectoryLoader
from langchain_huggingface import HuggingFaceEmbeddings
import model as md
# from dotenv import load_dotenv, find_dotenv

### TODO: does it work when it has to create a new chroma db?
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
current_question = "What is Data Science? List the di􀃠erences between supervised and unsupervised learning."
expl,results = md.question_explanation(current_question,embeddings)
print(expl)

c:\Users\raulg\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Reading Chroma DB...
Searching for similar questions...
Creating prompt...
Making chain...
Invoking explanation...
Storing answer in SQL...
Based on the provided context, here is the answer:

Data Science is a blend of various tools, algorithms, and machine learning principles with the goal to discover hidden patterns from the raw data. How is this different from what statisticians have been doing for years? The answer lies in the difference between explaining and predicting.

The differences between supervised and unsupervised learning are:

* Supervised Learning:
	+ Input data is labelled
	+ Uses a training data set
	+ Used for prediction
	+ Enables classification and regression
* Unsupervised Learning:
	+ Input data is unlabelled
	+ Uses the input data set
	+ Used for analysis
	+ Enables classification, density estimation, and dimension reduction


In [3]:
results

[Document(id='9bb207ab-b098-4aa4-aa44-876b0c7bb3d7', metadata={'source': 'data\\text_db\\raw\\100_ds_interview_questions.pdf', 'start_index': 1060}, page_content='Deep Learning Interview Questions\n\nBefore moving ahead, you may go through the recording of Data Science Interview Questions where our instructor has shared\n\nhis experience and expertise that will help you to crack any Data Science.\n\nData Science Interview Questions | Edureka\n\nData Science Interview Questions | Data Science Tutorial | Data Science I Data Science Interview Questions | Data Science Tutorial | Data Science I\n\nBASIC DATA SCIENCE INTERVIEW QUESTIONS\n\nQ1. What is Data Science? List the di\x00erences between supervised and unsupervised learning.\n\nData Science is a blend of various tools, algorithms, and machine learning principles with the goal to discover hidden patterns\n\nfrom the raw data. How is this di\x00erent from what statisticians have been doing for years?\n\nThe answer lies in the di\x00ere

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import model as md


In [7]:
CHROMA_PATH = "data/text_db/chroma"
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [8]:
current_question = "What is Data Science? List the di􀃠erences between supervised and unsupervised learning."
expl, results = md.question_explanation(current_question,embeddings)
print(expl)
print("="*50)
print(results)

Reading Chroma DB...
Searching for similar questions...
Creating prompt...
Making chain...
Invoking explanation...
Storing answer in SQL...
I apologize, but there is no context provided, so I cannot answer the question. The context is simply an empty list followed by a dash. Can you please provide more context or information so I can better understand and provide a helpful response?
[]


In [7]:
current_question = "What is Data Science? List the di􀃠erences between supervised and unsupervised learning."
# chroma_db = Chroma(persist_directory=CHROMA_PATH,embedding_function=embeddings)
results = chroma_db.similarity_search(current_question,k=3)
context_text = "\n\n---\n\n".join([doc.page_content for doc in results])    
print(context_text)

Deep Learning Interview Questions

Before moving ahead, you may go through the recording of Data Science Interview Questions where our instructor has shared

his experience and expertise that will help you to crack any Data Science.

Data Science Interview Questions | Edureka

Data Science Interview Questions | Data Science Tutorial | Data Science I Data Science Interview Questions | Data Science Tutorial | Data Science I

BASIC DATA SCIENCE INTERVIEW QUESTIONS

Q1. What is Data Science? List the di erences between supervised and unsupervised learning.

Data Science is a blend of various tools, algorithms, and machine learning principles with the goal to discover hidden patterns

from the raw data. How is this di erent from what statisticians have been doing for years?

The answer lies in the di erence between explaining and predicting.

The di erences between supervised and unsupervised learning are as follows;

---

BASIC DATA SCIENCE INTERVIEW QUESTIONS

Q1. What is Data Science? Li

In [8]:
results

[Document(id='7c02c9d7-a00a-485c-b574-a03ad2c2a49a', metadata={'source': 'data\\text_db\\raw\\100_ds_interview_questions.pdf', 'start_index': 1060}, page_content='Deep Learning Interview Questions\n\nBefore moving ahead, you may go through the recording of Data Science Interview Questions where our instructor has shared\n\nhis experience and expertise that will help you to crack any Data Science.\n\nData Science Interview Questions | Edureka\n\nData Science Interview Questions | Data Science Tutorial | Data Science I Data Science Interview Questions | Data Science Tutorial | Data Science I\n\nBASIC DATA SCIENCE INTERVIEW QUESTIONS\n\nQ1. What is Data Science? List the di\x00erences between supervised and unsupervised learning.\n\nData Science is a blend of various tools, algorithms, and machine learning principles with the goal to discover hidden patterns\n\nfrom the raw data. How is this di\x00erent from what statisticians have been doing for years?\n\nThe answer lies in the di\x00ere

In [9]:
expl, results = md.question_explanation(current_question,embeddings)
print(expl)
print("="*50)
print(results)

Reading Chroma DB...
Searching for similar questions...
Creating prompt...
Making chain...
Invoking explanation...
Storing answer in SQL...
According to the context, Data Science is a blend of various tools, algorithms, and machine learning principles with the goal to discover hidden patterns from the raw data. This is different from what statisticians have been doing for years, as it involves predicting rather than explaining.

The differences between supervised and unsupervised learning are:

* Supervised Learning:
	+ Input data is labelled.
	+ Uses a training data set.
	+ Used for prediction.
	+ Enables classification and regression.
* Unsupervised Learning:
	+ Input data is unlabelled.
	+ Uses the input data set.
	+ Used for analysis.
	+ Enables classification, density estimation, and dimension reduction.
[Document(id='7c02c9d7-a00a-485c-b574-a03ad2c2a49a', metadata={'source': 'data\\text_db\\raw\\100_ds_interview_questions.pdf', 'start_index': 1060}, page_content='Deep Learning In

In [1]:
import model as md
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma # from langchain_chroma import Chroma ### This MUST go here. For some reason it doesn't detect the chroma_db if it isn't inside the function.

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
current_question = "What is an example of a dataset with a non-Gaussian distribution?"
expl= md.question_explanation(current_question,embeddings)
print(expl)

c:\Users\raulg\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Reading Chroma DB...
Searching for similar questions...
Error during similarity search: Embedding dimension 384 does not match collection dimensionality 768
None


In [7]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

CHROMA_PATH = "data/text_db/chroma"
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
us_quest = "What is an example of a dataset with a non-Gaussian distribution?"
try:
    chroma_db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
    results = chroma_db.similarity_search(us_quest, k=5)
    print("Results:", results)
except Exception as e:
    print("Test failed:", e)

Results: [Document(id='bc84ff43-7f9a-4506-bd96-50d0b39cb268', metadata={'source': 'data\\text_db\\raw\\ds_interview_questions.pdf', 'start_index': 15519}, page_content='What is very important is that the Normal distri-\n\ndata is handling missing values. These are repre-\n\nbution is symmetrical around its mean, with a con-\n\nsented by NA in R. Impossible values (division by 0,\n\ncentration of observations around the mean. More-\n\nfor example) are represented by NAN (not a number).\n\nover, its mean, median and mode are the same.\n\nFinally, you should get an extra point if you mention\n\nthat 95% of the data points from a Normal distribu-\n\ntion are located within 2 standard deviations from\n\nthe mean, and 99.7% of the data points are located\n\nwithin 3 standard deviations from the mean.\n\n9. What is an example of a non-Gaussian distribution? First, it may make sense to research what is a\n\ndataset with a\n\nNow, you may be also expected to give an exam-\n\nGaussian distributi

# Pruebas 22/01

In [1]:
from model import question_explanation
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
us_quest = "What is an example of a dataset with a non-Gaussian distribution?"
expl, res = question_explanation(embeddings,us_quest)
print(expl)

c:\Users\raulg\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
### I try to create the vectorstore following this page: https://python.langchain.com/docs/integrations/vectorstores/chroma/#add-items-to-vector-store
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import model as md

CHROMA_PATH = "./data/text_db/chroma"
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

c:\Users\raulg\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
### Creation of Chroma DB and addition of docs
chunks = md.load_split_documents()
vector_store = Chroma(collection_name="doc_library", embedding_function=embeddings, persist_directory=CHROMA_PATH)
vector_store.add_documents(documents=chunks)
results = vector_store.similarity_search("What is an example of a dataset with a non-Gaussian distribution?", k=3)
results

c:\Users\raulg\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(id='70e38d36-1af4-4859-8da1-d22c79f59cdb', metadata={'source': 'data\\text_db\\raw\\ds_interview_questions.pdf'}, page_content='which are mainly geographical, but the overall pat-\n\ncounterexample.\n\ntern is such.\n\n10\n\nDATA SCIENTIST INTERVIEW QUESTIONS / GENERAL DATA SCIENTIST INTERVIEW QUESTIONS\n\nOne of the popular non-Gaussian instances is the\n\ndistribution of the household income in the USA .'),
 Document(id='e5ee66a6-abce-4193-bbab-2eeffe052ff3', metadata={'source': 'data\\text_db\\raw\\ds_interview_questions.pdf'}, page_content='tion are located within 2 standard deviations from\n\nthe mean, and 99.7% of the data points are located\n\nwithin 3 standard deviations from the mean.\n\n9. What is an example of a non-Gaussian distribution? First, it may make sense to research what is a\n\ndataset with a'),
 Document(id='4cbde9d0-3fe9-46d7-b60a-2436497e0c81', metadata={'source': 'data\\text_db\\raw\\ds_interview_questions.pdf'}, page_content='distributed it is going 

In [2]:
vector_store = Chroma(collection_name="doc_library", embedding_function=embeddings, persist_directory=CHROMA_PATH)
results = vector_store.similarity_search("What is Data Science", k=3)
results

[Document(id='cd43753c-7fa3-4417-9e57-9bbafb6e9d4f', metadata={'source': 'data\\text_db\\raw\\100_ds_interview_questions.pdf'}, page_content='Data Science is a blend of various tools, algorithms, and machine learning principles with the goal to discover hidden patterns\n\nfrom the raw data. How is this di\x00erent from what statisticians have been doing for years?\n\nThe answer lies in the di\x00erence between explaining and predicting.'),
 Document(id='1dfdac74-167f-42db-aee4-7daff226045a', metadata={'source': 'data\\text_db\\raw\\ds_interview_questions.pdf'}, page_content='‘If we think about data science as a field, we can\n\nming and often does not require statistical knowledge.\n\nidentify several pillars it is built upon: Mathematics,\n\nThat’s why data engineers and data architects exist.\n\nProbability, Statistics, Economics, Programming, Data'),
 Document(id='44895806-f35f-4be7-a8f4-278c8709d49d', metadata={'source': 'data\\text_db\\raw\\ds_interview_questions.pdf'}, page_conte

In [1]:
from model import question_explanation
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
us_quest = "What is an example of a dataset with a non-Gaussian distribution?"
expl, res = question_explanation(embeddings,us_quest)
print(expl)

c:\Users\raulg\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Loading documents...
Searching for similar questions...


: 

In [ ]:
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

CHROMA_PATH = "./data/text_db/chroma"
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
us_quest = "What is an example of a dataset with a non-Gaussian distribution?"


load_dotenv(override=True)
groq_api_key = os.getenv("GROQ_API_KEY")
model = ChatGroq(model="llama3-8b-8192")



def question_explanation(embeddings, current_question):
    print("Loading documents...")
    vector_store = Chroma(collection_name="doc_library", embedding_function=embeddings, persist_directory=CHROMA_PATH)
    print("Searching for similar questions...")
    results = vector_store.similarity_search(current_question, k=3)
    print("Creating prompt and invoking model...")
    prompt_template = ChatPromptTemplate.from_template("""
    Answer the question based only on the following context:

    {context}

    ---

    Answer the question based on the above context: {question}
    """)
    prompt = prompt_template.format(context=results, question=current_question)
    response = model.invoke(prompt)
    return response, results

c:\Users\raulg\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'ChatGroq' is not defined

In [ ]:
expl, res = question_explanation(embeddings,us_quest)
print(expl)

Loading documents...
Searching for similar questions...
Creating prompt and invoking model...


NameError: name 'model' is not defined